In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display, Javascript

def disable_scroll():
    display(Javascript("""
    IPython.OutputArea.prototype._should_scroll = function(lines) {
        return false;
    }
    """))

disable_scroll()

<IPython.core.display.Javascript object>

In [3]:
from KuhnPoker import KuhnPokerModel, KuhnPokerInfoSet, Card, TensorModel, gen_tree_hist, PASS, ADD_CHIP
from TreeVisualizer import plot_trees
from ISMCTS import Constants

import numpy as np
import pickle
import ipywidgets as widgets
from IPython.display import display
import torch

np.set_printoptions(precision=2)

In [4]:
vmodel = torch.load('exp_trivial3/vmodel/vmodel-600.pt')
pmodel = torch.load('exp_trivial3/pmodel/pmodel-600.pt')
model = TensorModel(vmodel, pmodel, learn_trivial=True)

# model = KuhnPokerModel(1/3, 0.34)

# info_set = KuhnPokerInfoSet([PASS, ADD_CHIP], [Card.QUEEN, None]) # Alice
info_set = KuhnPokerInfoSet([PASS], [None, Card.JACK]) # Bob
# info_set = KuhnPokerInfoSet([], [Card.QUEEN, None]) # Alice first trivial move

Constants.c_PUCT = 1.0
Constants.Dirichlet_ALPHA = 0.1

trees_hist = gen_tree_hist(model, info_set, dirichlet=True, iter=1000, eps=0.0)

{0: 0.181, 1: 0.819}


In [5]:
trees_hist[999][0].root.dirichlet_draw

array([7.35e-05, 1.00e+00])

In [6]:
slider = widgets.IntSlider(min=0, max=len(trees_hist)-1, step=1, description='Hist Index')

def update_plot(index):
    plot_trees(trees_hist[index])
    
widgets.interactive(update_plot, index=slider)

interactive(children=(IntSlider(value=0, description='Hist Index', max=1000), Output()), _dom_classes=('widget…